<br>

# ETA Linear Array

<br>

## environmental

<br>

In [132]:
#load methods
from scipy.io import loadmat
import pandas as pd

In [133]:
#load .mat into python environment
linArrayPath = "/home/bonzilla/Documents/smilodonCub_fossils/MATLAB/linArrayWorkspace.mat"
linArrayMat = loadmat( linArrayPath )

In [134]:
print( type( linArrayMat ) )
linArrayMat.keys()

<class 'dict'>


dict_keys(['__header__', '__version__', '__globals__', 'FP09', 'FP09_ind', 'FP09_ts', 'FP09_ts_step', 'FP10', 'FP10_ind', 'FP10_ts', 'FP10_ts_step', 'FP11', 'FP11_ind', 'FP11_ts', 'FP11_ts_step', 'FP12', 'FP12_ind', 'FP12_ts', 'FP12_ts_step', 'FP13', 'FP13_ind', 'FP13_ts', 'FP13_ts_step', 'FP14', 'FP14_ind', 'FP14_ts', 'FP14_ts_step', 'FP15', 'FP15_ind', 'FP15_ts', 'FP15_ts_step', 'FP16', 'FP16_ind', 'FP16_ts', 'FP16_ts_step', 'SPK09a', 'SPK09i', 'SPK10a', 'SPK10i', 'SPK11a', 'SPK11i', 'SPK12i', 'SPK13a', 'SPK13i', 'SPK14a', 'SPK14i', 'SPK15a', 'SPK15i', 'SPK16a', 'SPK16i', 'Start', 'Stop', 'Strobed00001', 'Strobed00003', 'Strobed00005', 'Strobed00006', 'Strobed00007', 'Strobed00009', 'Strobed00010', 'Strobed00013', 'Strobed00014', 'Strobed00016', 'Strobed00017', 'Strobed00018', 'Strobed00030', 'Strobed00040', 'Strobed00041', 'Strobed00043', 'Strobed00046', 'Strobed00050', 'Strobed00100', 'Strobed00102', 'Strobed00131', 'Strobed00161', 'Strobed00235', 'Strobed00360', 'Strobed00400', 'S

<br>

## reshape mat struct

reshape the imported matlab structure tosomethng more pythonic  
the matlab struct for the linear array is imported by `loadmat()` as a python dictionary. to facilitate further analysis, we will massage the data into a more workable form as a pandas dataframe...

<br>

### understand the dictionary fields

<br>

In [135]:
# the delayedSaccIntervals field
delayedSaccade_dat = linArrayMat[ 'delayedSaccIntervals' ]
delayedSaccade_DF = pd.DataFrame( data = delayedSaccade_dat, columns = [ 'start','end' ] )
delayedSaccade_DF.head()

,start,end
0,0.000,21.2835
1,21.832,24.9940
2,25.436,28.7080
3,29.250,32.3125
4,32.854,36.1360


In [136]:
# the catSearchPHYSIOLIntervals feld
catSearchPHYSIOL_dat = linArrayMat[ 'catSearchPHYSIOLIntervals' ]
catSearchPHYSIOL_DF = pd.DataFrame( data = catSearchPHYSIOL_dat, columns = [ 'start','end' ] )
catSearchPHYSIOL_DF.head()

,start,end
0,362.9720,366.9035
1,367.4555,370.5080
2,371.0595,374.5520
3,375.0975,378.5905
4,379.1320,383.3935


In [137]:
# spiking data for channel 9
spike09_dat = linArrayMat[ 'SPK09a' ]
print( len( spike09_dat ) )
print( spike09_dat[0:10], )

18737
[[0.064 ]
 [0.19  ]
 [0.2495]
 [0.5405]
 [0.5545]
 [0.582 ]
 [0.634 ]
 [0.7025]
 [1.082 ]
 [1.3005]]


In [138]:
# field potnetial data for channel 9
FP09_dat = linArrayMat[ 'FP09' ]
print( len( FP09_dat ) )
print( FP09_dat[0:10] )

980450
[[ 0.00288222]
 [-0.0024075 ]
 [-0.00837538]
 [-0.00613742]
 [-0.00891791]
 [-0.01322428]
 [-0.01698812]
 [-0.01946343]
 [-0.02088759]
 [-0.02034505]]


In [139]:
# a dataframe that holds columns names for all channels with spike data    
allKeys = pd.DataFrame( linArrayMat.keys(), columns = [ 'channels' ] )
spikeKeys_IDX = allKeys['channels'].str.startswith( 'SPK' )
spikeKeys = allKeys[ spikeKeys_IDX ]
print( len( spikeKeys ) )
spikeKeys.head()

15


,channels
35,SPK09a
36,SPK09i
37,SPK10a
38,SPK10i
39,SPK11a


In [140]:
# some helper functions to help format the linear array dictionary
def getStartEndTimestanps( array_DF, task ):
    """
    return the trial start and end timestamps for a task
    """
    task_dat = array_DF[ task ]
    task_DF = pd.DataFrame( data = task_dat, 
                           columns = [ 'start','end' ] )
    return task_DF

def formatStrobedEventSTR( eventmarker_int ):
    #make a string for the strobed event column label
    numZeros = 12 - 7 - len( str( eventmarker_int ) )
    strobedLabel = 'Strobed' + '0'*numZeros + str( eventmarker_int )
    return strobedLabel

def getChannelTrialData( array_DF, task, channel ):
    """
    generates a list for each trial within a pandas series for a specific task.
    array_DF: pandas dataframe with array data
    task: string for task batched data intervals
    channel: string for a plexon channel (e.g. SPK or FP)
    returns a pandas series of lists for trial data of a given channel
    """
    # a DF of all the spike timestamps for `channel`
    channelDat = array_DF[ channel ]
    channelDat_DF = pd.DataFrame( data = channelDat, 
                                 columns = [ 'timestamps' ] )
    # a DF for all the start/stop timestamps for each trial for `task`
    task_DF = getStartEndTimestanps( array_DF, task )
    numTrials = len( task_DF[ 'start' ] )
    timeStampLists = pd.Series([], dtype= 'float64')
    
    #for each trial, get the spike timestamps inbetween the trial
    #start/end and format as a list
    for aTrial in range( numTrials ):
        startTime = task_DF[ 'start' ][ aTrial ]
        endTime = task_DF[ 'end' ][ aTrial ]
        trialTimeStamps_IDX = channelDat_DF[ 'timestamps' ].between( startTime, endTime )
        trialTimeStamps = channelDat_DF[ trialTimeStamps_IDX ]
        listoflists_TS = trialTimeStamps.values.tolist()  
        timeStampLists[ aTrial ] = [item for sublist in listoflists_TS for item in sublist]
    # return a pandas.core.series.Series where each field is a list
    # of spike timestamps for each trial of task for a given channel
    return timeStampLists

def getDataTypeNames( array_DF, neuralDatType ):
    """
    returns a pandas dataframe with a feature of strings, one for each 
    plexon channel to be collected on
    array_DF: pandas dataframe with array data
    neuralDatType: (string) either 'SPK' or 'FP'. error is thrown for other values
    """
    allKeys = pd.DataFrame( array_DF.keys(), columns = [ 'channels' ] )
    typeKeys_IDX = allKeys['channels'].str.startswith( neuralDatType )
    dataTypeNames = allKeys[ typeKeys_IDX ]
    return dataTypeNames


def getMultiChannelTrialData( array_DF, dataType, task ):
    """
    given data and a specific task, returns a dataframe with a row for each 
    trial a feature for start and end timestamps and features for each channel 
    queried. channel features hold a list of data (e.g. spikes) for each trial
    array_DF: pandas dataframe with array data
    dataType: (string) SPK or FP 
    plexon channel to be collected on
    task: string for task batched data intervals
    """
    selectData_df = getDataTypeNames( array_DF, dataType )
    task_DF = getStartEndTimestanps( array_DF, task )

    #for each item in channel_df:
    for aChannel in range( len( selectData_df ) ):    
        #getTrialTimestamps
        channel = selectData_df[ 'channels' ].values[ aChannel ]
        channelTrialDat = getChannelTrialData( array_DF, task, channel )
        #print( type( channelTrialDat ) )
        task_DF[ channel ] = channelTrialDat
        #add as a new feature to trialDat_DF
    return task_DF

def addEventMarker( array_DF, task_DF, eventmarkers, task ):
    """
    takes in array dataframe and finds the timestamps for a specified
    eventmarker for each trial to be added as a feature column to a 
    dataframe of trial data (i.e. output from getMultiChannelTimestamps())
    array_DF: pandas dataframe generated from a matlab structure summarizing array data
    trialDat_DF: pandas dataframe with data by trial (output from getMultiChannelTrialData())
    eventmarker: int, specifies which eventmarker to parse from array_DF
    """
    
    for eventmarker in eventmarkers:
        #make a string for the strobed event column label
        strobedLabel = formatStrobedEventSTR( eventmarker )

        #check that the eventmarker is present
        colNames = pd.DataFrame( array_DF.keys(), columns = [ 'keys' ] )
        if colNames[ 'keys' ].str.contains( strobedLabel ).any():
            taskEventMarkers = getChannelTrialData( array_DF, task, strobedLabel )
        else:
            raise Exception( 'Sorry, eventmarker not present\nbe sure to enter \
                        only the integer value of the event\nex: CORRECT: 14 \
                        INCORRECT: 00014, Strobed00014')
        task_DF[ strobedLabel ] = taskEventMarkers    
    return task_DF

In [141]:
# demonstrate getChannelTrialData()
# generates a pandas series witha row for each trial holding a list of spike times
SPK09a_spikedat = getChannelTrialData( linArrayMat, 'delayedSaccIntervals', 'SPK09a')
SPK09a_spikedat[0:5]

0    [0.064, 0.19, 0.2495, 0.5405, 0.5545, 0.582, 0...
1    [21.9155, 22.0605, 22.519, 22.8845, 22.9575, 2...
2    [25.9435, 26.0485, 26.1345, 26.1595, 26.534, 2...
3    [29.518, 29.954, 30.0885, 30.2455, 30.3545, 30...
4    [32.897, 32.9765, 33.289, 33.5505, 33.6065, 33...
dtype: object

In [142]:
# demonstrate getMultiChannelTrialData()
# generates a pandas dataframe with a row for each trial, start/end features and columns for each channeldata
dSacctrials = getMultiChannelTrialData( linArrayMat, 'SPK', 'delayedSaccIntervals' )
dSacctrials.head()

,start,end,SPK09a,SPK09i,SPK10a,SPK10i,SPK11a,SPK11i,SPK12i,SPK13a,SPK13i,SPK14a,SPK14i,SPK15a,SPK15i,SPK16a,SPK16i
0,0.000,21.2835,"[0.064, 0.19, 0.2495, 0.5405, 0.5545, 0.582, 0...","[0.013, 0.0145, 0.1245, 0.153, 0.448, 0.848, 2...","[0.487, 1.238, 1.4395, 1.45, 1.588, 1.667, 1.9...","[5.6615, 12.6, 18.806, 18.813]","[0.003, 0.011, 0.013, 0.017, 0.019, 0.0265, 0....","[0.0235, 0.028, 0.051, 0.067, 0.0915, 0.1115, ...","[0.0005, 0.001, 0.003, 0.011, 0.0155, 0.0235, ...",[],"[0.001, 0.006, 0.009, 0.01, 0.013, 0.0145, 0.0...","[0.003, 0.0075, 0.0105, 0.019, 0.0265, 0.0325,...","[0.0025, 0.0055, 0.0365, 0.068, 0.118, 0.1815,...","[0.303, 0.347, 2.087, 2.578, 2.756, 8.532, 8.5...",[],"[0.004, 0.0065, 0.0075, 0.0365, 0.037, 0.045, ...",[]
1,21.832,24.9940,"[21.9155, 22.0605, 22.519, 22.8845, 22.9575, 2...","[21.858, 22.4455, 22.758, 22.809, 23.343, 23.3...","[22.436, 22.438, 22.4405, 22.445, 22.448, 22.4...",[],"[21.8505, 21.8895, 21.895, 21.9085, 21.911, 21...","[21.844, 21.874, 21.9035, 21.9355, 21.961, 22....","[21.8325, 21.838, 21.841, 21.8415, 21.8435, 21...",[],"[21.867, 21.898, 21.922, 21.997, 22.2705, 22.3...","[21.844, 21.8695, 21.924, 21.9515, 21.9625, 22...","[21.8755, 22.002, 22.425, 22.4285, 22.4295, 22...","[22.478, 22.692, 22.7295, 23.142, 23.153, 23.1...","[23.009, 23.197]","[21.932, 22.0095, 22.0405, 22.089, 22.159, 22....",[]
2,25.436,28.7080,"[25.9435, 26.0485, 26.1345, 26.1595, 26.534, 2...","[26.0355, 26.1315, 26.2155, 26.4875, 26.877, 2...","[25.5225, 25.5685, 25.9695, 26.1285, 26.1315, ...","[26.1435, 27.094, 27.097]","[25.496, 25.508, 25.5145, 25.516, 25.5195, 25....","[25.473, 25.493, 25.5355, 25.559, 25.5605, 26....","[25.4385, 25.448, 25.454, 25.4575, 25.46, 25.4...",[],"[25.493, 25.517, 25.533, 25.536, 25.537, 25.54...","[25.4475, 25.645, 25.649, 25.6595, 25.73, 25.7...","[25.503, 25.6845, 25.89, 26.0285, 26.0755, 26....","[26.3605, 26.3815, 26.5835, 26.667, 26.6855, 2...",[],"[25.6065, 25.6455, 25.6525, 25.6895, 25.7195, ...",[]
3,29.250,32.3125,"[29.518, 29.954, 30.0885, 30.2455, 30.3545, 30...","[29.9335, 29.9395, 30.771, 30.801, 31.0955, 31...","[29.9335, 29.95, 29.9555, 30.0295, 30.1905, 30...",[],"[29.252, 29.2625, 29.27, 29.272, 29.275, 29.31...","[29.258, 29.2655, 29.274, 29.3615, 29.3645, 29...","[29.2515, 29.2525, 29.253, 29.2555, 29.258, 29...",[],"[29.252, 29.2565, 29.376, 29.4095, 29.4155, 29...","[29.252, 29.2535, 29.266, 29.3385, 29.417, 29....","[29.2505, 29.255, 29.2565, 29.2585, 29.2605, 2...","[30.2355, 30.4665, 30.4985, 30.51, 30.536, 30....",[],"[29.254, 29.26, 29.263, 29.3215, 29.3525, 29.4...",[29.9425]
4,32.854,36.1360,"[32.897, 32.9765, 33.289, 33.5505, 33.6065, 33...","[32.9485, 33.5995, 33.7015, 33.9385, 34.5225, ...","[32.863, 32.945, 32.9635, 33.595, 33.597, 33.6...",[33.6065],"[32.8595, 32.867, 32.878, 32.8795, 32.8805, 32...","[32.8835, 32.888, 32.925, 32.9255, 32.9395, 32...","[32.8555, 32.857, 32.858, 32.859, 32.864, 32.8...",[],"[32.8735, 32.876, 32.896, 32.904, 32.9075, 32....","[32.886, 32.887, 32.896, 32.904, 32.911, 32.95...","[32.9315, 32.9405, 32.9485, 32.9875, 33.4255, ...","[33.658, 33.867, 34.013, 34.0425, 34.092, 34.2...",[],"[32.8765, 32.907, 32.9215, 32.945, 32.9875, 33...",[33.6075]


In [143]:
addStrobes = dSacctrials.copy()
addEventMarker( linArrayMat, addStrobes, [1, 30, 40, 666, 100, 2018], 'delayedSaccIntervals' )
addStrobes.head()

,start,end,SPK09a,SPK09i,SPK10a,SPK10i,SPK11a,SPK11i,SPK12i,SPK13a,...,SPK15a,SPK15i,SPK16a,SPK16i,Strobed00001,Strobed00030,Strobed00040,Strobed00666,Strobed00100,Strobed02018
0,0.000,21.2835,"[0.064, 0.19, 0.2495, 0.5405, 0.5545, 0.582, 0...","[0.013, 0.0145, 0.1245, 0.153, 0.448, 0.848, 2...","[0.487, 1.238, 1.4395, 1.45, 1.588, 1.667, 1.9...","[5.6615, 12.6, 18.806, 18.813]","[0.003, 0.011, 0.013, 0.017, 0.019, 0.0265, 0....","[0.0235, 0.028, 0.051, 0.067, 0.0915, 0.1115, ...","[0.0005, 0.001, 0.003, 0.011, 0.0155, 0.0235, ...",[],...,"[0.303, 0.347, 2.087, 2.578, 2.756, 8.532, 8.5...",[],"[0.004, 0.0065, 0.0075, 0.0365, 0.037, 0.045, ...",[],[17.961],[18.7475],[19.306],[19.8995],[20.949],[17.8095]
1,21.832,24.9940,"[21.9155, 22.0605, 22.519, 22.8845, 22.9575, 2...","[21.858, 22.4455, 22.758, 22.809, 23.343, 23.3...","[22.436, 22.438, 22.4405, 22.445, 22.448, 22.4...",[],"[21.8505, 21.8895, 21.895, 21.9085, 21.911, 21...","[21.844, 21.874, 21.9035, 21.9355, 21.961, 22....","[21.8325, 21.838, 21.841, 21.8415, 21.8435, 21...",[],...,"[22.478, 22.692, 22.7295, 23.142, 23.153, 23.1...","[23.009, 23.197]","[21.932, 22.0095, 22.0405, 22.089, 22.159, 22....",[],[],[22.3725],[23.0395],[23.6325],[24.6655],[]
2,25.436,28.7080,"[25.9435, 26.0485, 26.1345, 26.1595, 26.534, 2...","[26.0355, 26.1315, 26.2155, 26.4875, 26.877, 2...","[25.5225, 25.5685, 25.9695, 26.1285, 26.1315, ...","[26.1435, 27.094, 27.097]","[25.496, 25.508, 25.5145, 25.516, 25.5195, 25....","[25.473, 25.493, 25.5355, 25.559, 25.5605, 26....","[25.4385, 25.448, 25.454, 25.4575, 25.46, 25.4...",[],...,"[26.3605, 26.3815, 26.5835, 26.667, 26.6855, 2...",[],"[25.6065, 25.6455, 25.6525, 25.6895, 25.7195, ...",[],[25.4545],[26.064],[26.581],[27.383],[28.432],[]
3,29.250,32.3125,"[29.518, 29.954, 30.0885, 30.2455, 30.3545, 30...","[29.9335, 29.9395, 30.771, 30.801, 31.0955, 31...","[29.9335, 29.95, 29.9555, 30.0295, 30.1905, 30...",[],"[29.252, 29.2625, 29.27, 29.272, 29.275, 29.31...","[29.258, 29.2655, 29.274, 29.3615, 29.3645, 29...","[29.2515, 29.2525, 29.253, 29.2555, 29.258, 29...",[],...,"[30.2355, 30.4665, 30.4985, 30.51, 30.536, 30....",[],"[29.254, 29.26, 29.263, 29.3215, 29.3525, 29.4...",[29.9425],[],[29.881],[30.3975],[31.0155],[32.057],[]
4,32.854,36.1360,"[32.897, 32.9765, 33.289, 33.5505, 33.6065, 33...","[32.9485, 33.5995, 33.7015, 33.9385, 34.5225, ...","[32.863, 32.945, 32.9635, 33.595, 33.597, 33.6...",[33.6065],"[32.8595, 32.867, 32.878, 32.8795, 32.8805, 32...","[32.8835, 32.888, 32.925, 32.9255, 32.9395, 32...","[32.8555, 32.857, 32.858, 32.859, 32.864, 32.8...",[],...,"[33.658, 33.867, 34.013, 34.0425, 34.092, 34.2...",[],"[32.8765, 32.907, 32.9215, 32.945, 32.9875, 33...",[33.6075],[],[33.539],[34.2055],[34.815],[35.848],[]


<br>

### Aligning Neural Data

align the data by stimulus onset. 

<br>

In [185]:
stimOn = 30 #define the eventmarker for stimulus onset

# define a function to adjust timestamps by stimulus onset
def stimOnset( data_df, stimOnset_eventmarker, dataType ):
    """
    returns an array with neual data adjusted for the stimulus onset time
    """
    adjustdata_df = data_df.copy()
    
    #make a string for the strobed event column label
    strobedLabel = formatStrobedEventSTR( stimOnset_eventmarker )
    #print( strobedLabel )
    
    #make a list of data columns to adjust
    columns2adjust = getDataTypeNames( data_df, dataType )
    columns2adjust = columns2adjust.values.tolist() 
    columns2adjust = [item for sublist in columns2adjust for item in sublist]
    
    #for each column
    for column in columns2adjust:
        #print( column )
        for idx, alist in enumerate( data_df[ column ] ):
            trialStimOn_time = data_df[ strobedLabel ][idx]
            #print( idx, alist )
            adjustlist = [round( timestamp - trialStimOn_time, 4 ) for timestamp in alist]
            #print( idx, adjustlist )
            adjustdata_df[ column ][ idx ] = adjustlist
        
    return adjustdata_df

In [186]:
a = stimOnset( addStrobes, stimOn, 'SPK' )
a.head()

/home/bonzilla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,start,end,SPK09a,SPK09i,SPK10a,SPK10i,SPK11a,SPK11i,SPK12i,SPK13a,...,SPK15a,SPK15i,SPK16a,SPK16i,Strobed00001,Strobed00030,Strobed00040,Strobed00666,Strobed00100,Strobed02018
0,0.000,21.2835,"[-18.6835, -18.5575, -18.498, -18.207, -18.193...","[-18.7345, -18.733, -18.623, -18.5945, -18.299...","[-18.2605, -17.5095, -17.308, -17.2975, -17.15...","[-13.086, -6.1475, 0.0585, 0.0655]","[-18.7445, -18.7365, -18.7345, -18.7305, -18.7...","[-18.724, -18.7195, -18.6965, -18.6805, -18.65...","[-18.747, -18.7465, -18.7445, -18.7365, -18.73...",[],...,"[-18.4445, -18.4005, -16.6605, -16.1695, -15.9...",[],"[-18.7435, -18.741, -18.74, -18.711, -18.7105,...",[],[17.961],18.7475,[19.306],[19.8995],[20.949],[17.8095]
1,21.832,24.9940,"[-0.457, -0.312, 0.1465, 0.512, 0.585, 0.794, ...","[-0.5145, 0.073, 0.3855, 0.4365, 0.9705, 0.983...","[0.0635, 0.0655, 0.068, 0.0725, 0.0755, 0.1225...",[],"[-0.522, -0.483, -0.4775, -0.464, -0.4615, -0....","[-0.5285, -0.4985, -0.469, -0.437, -0.4115, -0...","[-0.54, -0.5345, -0.5315, -0.531, -0.529, -0.5...",[],...,"[0.1055, 0.3195, 0.357, 0.7695, 0.7805, 0.797,...","[0.6365, 0.8245]","[-0.4405, -0.363, -0.332, -0.2835, -0.2135, -0...",[],[],22.3725,[23.0395],[23.6325],[24.6655],[]
2,25.436,28.7080,"[-0.1205, -0.0155, 0.0705, 0.0955, 0.47, 0.79,...","[-0.0285, 0.0675, 0.1515, 0.4235, 0.813, 1.123...","[-0.5415, -0.4955, -0.0945, 0.0645, 0.0675, 0....","[0.0795, 1.03, 1.033]","[-0.568, -0.556, -0.5495, -0.548, -0.5445, -0....","[-0.591, -0.571, -0.5285, -0.505, -0.5035, -0....","[-0.6255, -0.616, -0.61, -0.6065, -0.604, -0.5...",[],...,"[0.2965, 0.3175, 0.5195, 0.603, 0.6215, 0.63, ...",[],"[-0.4575, -0.4185, -0.4115, -0.3745, -0.3445, ...",[],[25.4545],26.064,[26.581],[27.383],[28.432],[]
3,29.250,32.3125,"[-0.363, 0.073, 0.2075, 0.3645, 0.4735, 0.6005...","[0.0525, 0.0585, 0.89, 0.92, 1.2145, 1.4095]","[0.0525, 0.069, 0.0745, 0.1485, 0.3095, 0.82, ...",[],"[-0.629, -0.6185, -0.611, -0.609, -0.606, -0.5...","[-0.623, -0.6155, -0.607, -0.5195, -0.5165, -0...","[-0.6295, -0.6285, -0.628, -0.6255, -0.623, -0...",[],...,"[0.3545, 0.5855, 0.6175, 0.629, 0.655, 0.675, ...",[],"[-0.627, -0.621, -0.618, -0.5595, -0.5285, -0....",[0.0615],[],29.881,[30.3975],[31.0155],[32.057],[]
4,32.854,36.1360,"[-0.642, -0.5625, -0.25, 0.0115, 0.0675, 0.074...","[-0.5905, 0.0605, 0.1625, 0.3995, 0.9835, 1.33...","[-0.676, -0.594, -0.5755, 0.056, 0.058, 0.066,...",[0.0675],"[-0.6795, -0.672, -0.661, -0.6595, -0.6585, -0...","[-0.6555, -0.651, -0.614, -0.6135, -0.5995, -0...","[-0.6835, -0.682, -0.681, -0.68, -0.675, -0.67...",[],...,"[0.119, 0.328, 0.474, 0.5035, 0.553, 0.742, 0....",[],"[-0.6625, -0.632, -0.6175, -0.594, -0.5515, -0...",[0.0685],[],33.539,[34.2055],[34.815],[35.848],[]


NameError: name 'Strobed00030' is not defined